# Demo for Real Time Gait Asymmetry Detection Model Task

The current notebook is going to be a demo of the real time asymmetry detection model capabilities, while also being used as the initial steps for the creation of the pipeline of the deployed version.

In [ ]:
# Import Liraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.signal import find_peaks
from sklearn.linear_model import LogisticRegression
import time

In [ ]:
# ## Helper Functions
def asymmetry_index(left, right):
    return [(l - r) / (l + r) if (l + r) != 0 else 0 for l, r in zip(left, right)]

def symmetry_ratio(left, right):
    return [min(l, r) / max(l, r) if max(l, r) != 0 else 0 for l, r in zip(left, right)]

def summarize(lst):
    return float(np.mean(lst)) if len(lst) > 0 else 0

def simulate_gait_signal(amplitude, freq, phase_shift, noise, duration_sec=2.0, fs=100):
    t = np.linspace(0, duration_sec, int(fs * duration_sec))
    signal = amplitude * np.sin(2 * np.pi * freq * t + phase_shift)
    return signal + np.random.normal(0, noise, size=t.shape)

In [ ]:

def generate_simulated_window(healthy=True):
    fs = 100
    left = simulate_gait_signal(amplitude=50, freq=1.0, phase_shift=0.0, noise=2.0)
    if healthy:
        right = simulate_gait_signal(amplitude=50, freq=1.0, phase_shift=0.0, noise=2.0)
    else:
        right = simulate_gait_signal(amplitude=35, freq=1.0, phase_shift=0.1, noise=2.0)
    
    left_peaks, _ = find_peaks(left, height=10, distance=30)
    right_peaks, _ = find_peaks(right, height=10, distance=30)

    left_stride_times = np.diff(left_peaks)
    right_stride_times = np.diff(right_peaks)

    asym = asymmetry_index(left_stride_times, right_stride_times)
    sym = symmetry_ratio(left_stride_times, right_stride_times)

    return {
        "left": left,
        "right": right,
        "asymmetry_index": summarize(asym),
        "symmetry_ratio": summarize(sym)
    }

In [ ]:
# # Gait Asymmetry Detection — Real-Time Simulation Demo


# ## Load Trained Model from CSV Features
# Load the detection_asymmetry dataset and train a model
df = pd.read_csv("detection_asymmetry.csv")
df = df[df['label_lenient'] != 2].copy()

# Clean stringified list values
def clean_feature(x):
    try:
        return float(str(x).strip("[]").split(",")[0])
    except:
        return np.nan

df['gyro-asymmetry-stride-times'] = df['gyro-asymmetry-stride-times'].apply(clean_feature)
df['gyro-symmetry-ratio-stride-times'] = df['gyro-symmetry-ratio-stride-times'].apply(clean_feature)

features = ['gyro-asymmetry-stride-times', 'gyro-symmetry-ratio-stride-times']
X_train = df[features].values
y_train = df['label_lenient'].values

model = LogisticRegression(max_iter=1000)
model.fit(X_train, y_train)



# ## Generate Simulated Left and Right Gait Signals


# ## Run Simulation
def run_simulation(label="healthy"):
    healthy = label.lower() == "healthy"
    win = generate_simulated_window(healthy=healthy)

    print(f"Asymmetry Index (mean): {win['asymmetry_index']:.4f}")
    print(f"Symmetry Ratio (mean): {win['symmetry_ratio']:.4f}")

    features = np.array([[win['asymmetry_index'], win['symmetry_ratio']]])
    pred = model.predict(features)[0]
    label_pred = "ASYMMETRIC" if pred == 1 else "SYMMETRIC"

    print(f"Predicted Gait State: {label_pred}")

    # Plot
    plt.figure(figsize=(10, 4))
    plt.plot(win['left'], label='Left Shank')
    plt.plot(win['right'], label='Right Shank')
    plt.title(f"Simulated Gait — Prediction: {label_pred}")
    plt.xlabel("Time steps")
    plt.ylabel("Rotation (deg/s)")
    plt.legend()
    plt.grid(True)
    plt.show()

# ## Demo: Simulate Healthy and Stroke Gait
print("Simulating Healthy Gait")
run_simulation(label="healthy")

print("\\nSimulating Stroke (Asymmetric) Gait")
run_simulation(label="stroke")